## Etapa 4: Limpeza dos dados

Agora que possuimos os dois datasets que serão utilizados, um para treinar o modelo e o outro que queremos prever, vamos começar a tratar os dados para que a lingagem informal seja minimizada, e assim tentar melhorar a precisão do modelo. Também faremos a mesma limpeza em outros três datasets que utilizaremos para validação do modelo.

In [1]:
import pandas as pd 
import numpy as np
import re

In [2]:
# importando dataset de reviews
data = pd.read_csv('reviews_geral.csv', index_col=0)
data.head()

,content,score
0,"Gente, mudem pro design anterior. A interface ...",5
1,"Gosto muito desse banco, mas farei uma crítica...",5
2,"Até agora estou gostando, realmente essa nova ...",5
3,Adoro esse APP. Raramente apresenta alguma fal...,5
4,Simplesmente maravilhoso!! Eu nunca vi um banc...,5


In [3]:
# importando dataset de tweets
tweets = pd.read_csv('dataset.csv', index_col=0)
tweets.head()

,in_reply_to_screen_name,full_text,sentimento
0,santander_br,isso qualquer operacao que eu tente fazer em ...,NaN
1,santander_br,a atendente pagou pela segunda vez a fatura d...,NaN
2,Bancointer,pode trazer o meus,NaN
3,santander_br,nao estou conseguindo gerar cartao compra onl...,NaN
4,itau,mas foi e o setup foi bem facil muito bom,NaN


In [4]:
# importando datasets de validação
B2W = pd.read_csv('B2W-Reviews01.csv', delimiter=';')
tweets_aleatorios = pd.read_csv('NoThemeTweets.csv')
imdb = pd.read_csv('imdb-reviews-pt-br.csv')

C:\Users\LBKnaak\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
B2W.head(1)

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ


In [6]:
tweets_aleatorios.head(1)

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)


In [7]:
imdb.head(1)

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg


In [8]:
# função para trocar a nota 5 da avaliação por 1 (positivo) e a nota 1 por 0 (negativo)
def transforma_categoria(valor, categoria=1):
    if valor == categoria:
        return 0
    else:
        return 1

# função para trocar 'pos' por 1 (positivo) e 'neg' por 0 (negativo)
def transforma_categoria2(valor):
    if valor == 'neg':
        return 0
    else:
        return 1
    
# função para trocar 'Positivo' por 1 (positivo) e 'Negativo' por 0 (negativo)
def transforma_categoria3(valor):
    if valor == 'Negativo':
        return 0
    else:
        return 1

In [9]:
# aplicando a função de transformação da categoria
data['score'] = data.score.map(transforma_categoria)

imdb['sentiment'] = imdb.sentiment.map(transforma_categoria2)
tweets_aleatorios['sentiment'] = tweets_aleatorios.sentiment.map(transforma_categoria3)
#B2W['overall_rating'] = B2W.overall_rating.map(transforma_categoria)

In [10]:
# função para remover letras duplicadas
def removendo_duplicadas(texto):
    lista = []
    atual = ''
    anterior = ''
    for i in texto:
        if i != atual:
            lista.append(i)
            atual = i
        elif atual == 'r' or atual == 's':
            lista.append(i)
    return(''.join(lista))

#função para limpeza do texto
def limpa_texto(text):    
    text = str(text)
    text = re.sub('ã','a',text)
    text = re.sub('á','a',text)
    text = re.sub('à','a',text)
    text = re.sub('â','a',text)
    text = re.sub('ó','o',text)
    text = re.sub('õ','o',text)
    text = re.sub('ô','o',text)
    text = re.sub('é','e',text)
    text = re.sub('ê','e',text)
    text = re.sub('í','i',text)
    text = re.sub('ú','u',text)
    text = re.sub('ç','c',text)    
    text = re.sub(r'https?://\S+|www.\S+|@\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)
    text = re.sub(r'[0-9]', '', text)
    text = text.lower()
    text = removendo_duplicadas(text)
    return text

In [11]:
# aplicando a limpeza do texto nos datasets
data['content'] = data.content.map(limpa_texto)
tweets['full_text'] = tweets.full_text.map(limpa_texto)

#validacao
B2W['review_text'] = B2W.review_text.map(limpa_texto)
imdb['text_pt'] = imdb.text_pt.map(limpa_texto)
tweets_aleatorios['tweet_text'] = tweets_aleatorios.tweet_text.map(limpa_texto)

In [13]:
# removendo colunas que não serão utilizadas
tweets.drop(columns=['id_str','in_reply_to_status_id_str'], inplace=True)

B2W.drop(columns=['submission_date','reviewer_id','product_id','product_name','product_brand','site_category_lv1','site_category_lv2','review_title','recommend_to_a_friend','reviewer_birth_year','reviewer_gender','reviewer_state'], inplace=True)
imdb.drop((['text_en','id']), axis=1, inplace=True)
tweets_aleatorios.drop((['id','tweet_date','query_used']), axis=1, inplace=True)

In [14]:
# removendo vazios
tweets.drop(tweets[tweets['full_text'] == ''].index, inplace=True)
tweets.drop(tweets[tweets['full_text'] == ' '].index, inplace=True)
data.drop(data[data['content'] == ''].index, inplace=True)
data.drop(data[data['content'] == ' '].index, inplace=True)

# Validação
# removendo vazios e avaliações 2 a 4 do B2W
B2W.drop(B2W[B2W['review_text'] == ''].index, inplace=True)
B2W.drop(B2W[B2W['review_text'] == ' '].index, inplace=True)
B2W.drop(B2W[B2W['overall_rating'] == 2].index, inplace=True)
B2W.drop(B2W[B2W['overall_rating'] == 3].index, inplace=True)
B2W.drop(B2W[B2W['overall_rating'] == 4].index, inplace=True)
# removendo vazios
imdb.drop(imdb[imdb['text_pt'] == ''].index, inplace=True)
imdb.drop(imdb[imdb['text_pt'] == ' '].index, inplace=True)
tweets_aleatorios.drop(tweets_aleatorios[tweets_aleatorios['tweet_text'] == ''].index, inplace=True)
tweets_aleatorios.drop(tweets_aleatorios[tweets_aleatorios['tweet_text'] == ' '].index, inplace=True)


In [15]:
data.score.value_counts()

0    119253
1    117704
Name: score, dtype: int64

In [16]:
imdb.sentiment.value_counts()

0    24765
1    24694
Name: sentiment, dtype: int64

In [17]:
tweets_aleatorios.sentiment.value_counts()

0    522698
1    263094
Name: sentiment, dtype: int64

In [18]:
# balanceando o dataset
tweets_aleatorios.drop(tweets_aleatorios[tweets_aleatorios['sentiment'] == 0][:256000].index, inplace=True)

In [19]:
tweets_aleatorios.sentiment.value_counts()

0    266698
1    263094
Name: sentiment, dtype: int64

In [20]:
B2W.overall_rating.value_counts()

5    47897
1    27363
Name: overall_rating, dtype: int64

In [21]:
# balanceando o dataset
B2W.drop(B2W[B2W['overall_rating'] == 5][:20000].index, inplace=True)

In [22]:
# aplicando a função de transformação da categoria
B2W['overall_rating'] = B2W.overall_rating.map(transforma_categoria)

In [23]:
B2W.overall_rating.value_counts()

1    27897
0    27363
Name: overall_rating, dtype: int64

In [24]:
# verificando a existencia de nulos
tweets.full_text.isna().value_counts()

False    122524
Name: full_text, dtype: int64

In [25]:
# verificando a existencia de nulos
data.content.isna().value_counts()

False    236957
Name: content, dtype: int64

In [26]:
# verificando a existencia de nulos
imdb.text_pt.isna().value_counts()

False    49459
Name: text_pt, dtype: int64

In [27]:
# verificando a existencia de nulos
tweets_aleatorios.tweet_text.isna().value_counts()

False    529792
Name: tweet_text, dtype: int64

In [28]:
# verificando a existencia de nulos
B2W.review_text.isna().value_counts()

False    55260
Name: review_text, dtype: int64

In [29]:
# gravando dataset limpo
tweets.to_csv('dataset.csv')
data.to_csv('reviews_geral_limpo.csv')

# validação
imdb.to_csv('imdb_limpo.csv')
tweets_aleatorios.to_csv('tweets_aleatorios_limpo.csv')
B2W.to_csv('B2W_limpo.csv')

Para facilitar a aplicação dos modelos futuros, vamos deixar salvo um dataset de treino e teste.

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
# separa dados em treino e teste
train, test, _, _ = train_test_split(data, data['content'], test_size=0.3, random_state=42)

In [32]:
# gravando um dataset de treino
train.to_csv('train.csv')

In [33]:
# gravando um dataset de teste
test.to_csv('test.csv')